# 01_scrap

## Getting data from Kasikorn Research

In [ ]:
from bs4 import BeautifulSoup
import requests
import datetime
import pandas as pd
import json

In [ ]:
data = requests.get('https://www.kasikornresearch.com/th/analysis/Pages/index.aspx?s=ทุเรียน')
data

In [ ]:
s = BeautifulSoup(data.content, 'html.parser')

In [ ]:
publish_date = []
retrieve_date = []
title = []
content = []
source_url = []

In [ ]:
url_pre = 'https://www.kasikornresearch.com/'

for news in s.find_all('div', {'class': 'meta'}):
    link = url_pre + news.find('a')['href']
    source_url.append(link)
    
    topic = news.find('a').text
    title.append(topic)
    
    pub = news.find('p').text
    publish_date.append(pub)
    
    retrieve = str(datetime.date.today())
    retrieve_date.append(retrieve)

# print(publish_datetime)
# print(title) 
# print(source_url)

In [ ]:
for c in source_url:
    so = requests.get(c)
    no = BeautifulSoup(so.content, 'html.parser')
    
    for n in no.find_all('div', {'class', 'art-content entrycontent'}):
        a = n.text
        a = a.strip()
        a = a.replace('\u200b', '')
        a = a.replace('\xa0', '')
        a = a.replace('\r', '')
        a = a.replace('\n', '')
        a = a.replace('Scan QR Codeหมายเหตุรายงานวิจัยฉบับนี้จัดทำเพื่อเผยแพร่ทั่วไป โดยจัดทำขึ้นจากแหล่งข้อมูลต่างๆ ที่น่าเชื่อถือ แต่บริษัทฯ มิอาจรับรองความถูกต้อง ความน่าเชื่อถือ หรือความสมบูรณ์เพื่อใช้ในทางการค้าหรือประโยชน์อื่นใด บริษัทฯ อาจมีการเปลี่ยนแปลงข้อมูลได้ตลอดโดยไม่ต้องแจ้งให้ทราบล่วงหน้า ทั้งนี้ผู้ใช้ข้อมูลต้องใช้ความระมัดระวังในการใช้ข้อมูลต่างๆ ด้วยวิจารณญาณตนเองและรับผิดชอบในความเสี่ยงเองทั้งสิ้น บริษัทฯ จะไม่รับผิดต่อผู้ใช้หรือบุคคลใดในความเสียหายใดจากการใช้ข้อมูลดังกล่าว ข้อมูลในรายงานฉบับนี้จึงไม่ถือว่าเป็นการให้ความเห็น หรือคำแนะนำในการตัดสินใจทางธุรกิจ แต่อย่างใดทั้งสิ้น', '')
        content.append(a)

# print(content)

In [ ]:
df = pd.DataFrame([title, publish_date, content, source_url, retrieve_date]).transpose()
df.columns = ['title', 'publish_date', 'content', 'source_url', 'retrieve_date']
df

In [ ]:
df_json = df.to_dict('records')

In [ ]:
print(f'Total news : {len(df_json)}')

In [ ]:
with open('analysis_durians_kasikorn.json', 'w', encoding='utf8') as f:
    json.dump(df_json, f)

## Upload data to DataPlatform

In [ ]:
from dsmlibrary.datanode import DataNode

In [ ]:
dir_raw_id = 60

In [ ]:
datanode = DataNode()

In [ ]:
datanode.upload_file(directory_id=dir_raw_id, file_path='analysis_durians_kasikorn.json')